# Dataset Inspection

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function, division

from pathlib2 import Path

import numpy as np
import rospy

from vgn.dataset import Dataset, RandomAffine
from vgn.grasp import Grasp
from vgn.utils.transform import Rotation, Transform
from vgn_ros import vis

In [ ]:
DATASET_DIR = Path("../data/datasets/blocks")
AUGMENT = False

In [ ]:
# initialize a ros node to communicate with rviz
rospy.init_node("dataset_inspection")

In [ ]:
# load the dataset
dataset = Dataset(DATASET_DIR)

## Stats

In [ ]:
df = dataset.df

positives = df[df["label"] == 1]
negatives = df[df["label"] == 0]

print("Number of samples:", len(df.index))
print("Number of positives:", len(positives.index))
print("Number of negatives:", len(negatives.index))

## Visualize

In [ ]:
# select a random sample
i = np.random.choice(len(dataset))
tsdf, (rotation, width, label), index = dataset[i]

# transform to world frame
finger_depth = 0.05
size = 6 * finger_depth
voxel_size = size / 40
rotation = Rotation.from_quat(rotation)
position = index * voxel_size
width = width * voxel_size
grasp = Grasp(Transform(rotation, position), width)

# visualize
vis.clear()
vis.workspace(size)
vis.tsdf(tsdf.squeeze(), voxel_size)
vis.grasps([grasp], [float(label)], finger_depth)